# config

> Configuration for worker processes including restart policies, timeouts, and queue sizes.

In [ ]:
#| default_exp core.config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from enum import Enum

## Restart Policy

In [ ]:
#| export
class RestartPolicy(Enum):
    """Policy for restarting worker processes after failures."""
    NEVER = "never"  # Never restart workers
    ON_CANCELLATION = "on_cancellation"  # Restart only when job is cancelled
    ALWAYS = "always"  # Always restart on any failure
    BACKOFF = "backoff"  # Restart with exponential backoff

In [ ]:
# Example usage
RestartPolicy.ON_CANCELLATION

<RestartPolicy.ON_CANCELLATION: 'on_cancellation'>

## Worker Configuration

In [ ]:
#| export
@dataclass
class WorkerConfig:
    """Configuration for worker process behavior."""

    # Queue sizes
    request_queue_size: int = 0  # 0 = unlimited
    result_queue_size: int = 100  # Larger for streaming results
    response_queue_size: int = 10  # For synchronous command responses

    # Restart behavior
    restart_policy: RestartPolicy = RestartPolicy.ON_CANCELLATION
    max_restart_attempts: int = 3
    restart_backoff_base_seconds: float = 1.0  # Base delay for exponential backoff
    restart_backoff_max_seconds: float = 60.0  # Max delay for backoff

    # Worker pool (for future enhancement)
    max_workers: int = 1  # Currently only 1 is supported

    # Timeouts
    worker_start_timeout_seconds: float = 30.0
    reload_timeout_seconds: float = 30.0
    unload_timeout_seconds: float = 10.0
    shutdown_timeout_seconds: float = 5.0

    # Monitoring
    result_monitor_poll_interval_seconds: float = 0.5

    def __post_init__(self):
        """Validate configuration."""
        if self.max_workers > 1:
            raise NotImplementedError(
                f"Worker pools (max_workers > 1) are not yet implemented. "
                f"Current value: {self.max_workers}"
            )

        if self.max_workers < 1:
            raise ValueError(f"max_workers must be >= 1, got {self.max_workers}")

        if self.max_restart_attempts < 0:
            raise ValueError(f"max_restart_attempts must be >= 0, got {self.max_restart_attempts}")

        if self.restart_backoff_base_seconds <= 0:
            raise ValueError(
                f"restart_backoff_base_seconds must be > 0, got {self.restart_backoff_base_seconds}"
            )

In [ ]:
# Create default config
config = WorkerConfig()
config

WorkerConfig(request_queue_size=0, result_queue_size=100, response_queue_size=10, restart_policy=<RestartPolicy.ON_CANCELLATION: 'on_cancellation'>, max_restart_attempts=3, restart_backoff_base_seconds=1.0, restart_backoff_max_seconds=60.0, max_workers=1, worker_start_timeout_seconds=30.0, reload_timeout_seconds=30.0, unload_timeout_seconds=10.0, shutdown_timeout_seconds=5.0, result_monitor_poll_interval_seconds=0.5)

In [ ]:
# Create config with custom settings
config = WorkerConfig(
    restart_policy=RestartPolicy.BACKOFF,
    max_restart_attempts=5,
    result_queue_size=200
)
config

WorkerConfig(request_queue_size=0, result_queue_size=200, response_queue_size=10, restart_policy=<RestartPolicy.BACKOFF: 'backoff'>, max_restart_attempts=5, restart_backoff_base_seconds=1.0, restart_backoff_max_seconds=60.0, max_workers=1, worker_start_timeout_seconds=30.0, reload_timeout_seconds=30.0, unload_timeout_seconds=10.0, shutdown_timeout_seconds=5.0, result_monitor_poll_interval_seconds=0.5)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()